In [1]:
import math
import os
import time

# pip/conda installed
import dask.array as da
import fiona.transform
import fsspec
import pandas as pd
import xarray as xr
from netCDF4 import Dataset
from dask.distributed import as_completed
from dask.distributed import Client
from dask.distributed import performance_report
from dask_gateway import GatewayCluster

from utils import get_logger
from utils.dask import create_cluster
from utils.dask import upload_source

In [2]:
def reproject(lons, lats):
    dst_crs = "+proj=lcc +lat_1=25 +lat_2=60 +lat_0=42.5 +lon_0=-100 +x_0=0 +y_0=0 +ellps=WGS84 +units=m +no_defs"
    src_crs = "EPSG:4326"
    (x, y) = fiona.transform.transform(
        src_crs, dst_crs, lons, lats
    )
    return x, y

In [3]:
# fill with your account key
os.environ['AZURE_ACCOUNT_KEY'] = ""

In [4]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:33247")
client

Client Scheduler: tcp://127.0.0.1:33247 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 16 Memory: 33.68 GB


In [5]:
mean_src = fsspec.get_mapper(
    f"az://fia/daymet/na_40yr_monthly_means.zarr",
    account_name="usfs",
    account_key=os.environ['AZURE_ACCOUNT_KEY']
)
std_src = fsspec.get_mapper(
    f"az://fia/daymet/na_40yr_monthly_std_devs.zarr",
    account_name="usfs",
    account_key=os.environ['AZURE_ACCOUNT_KEY']
)

In [11]:
pt_means_dest = fsspec.get_mapper(
    f"az://fia/daymet/fia_40yr_monthly_means.zarr",
    account_name="usfs",
    account_key=os.environ['AZURE_ACCOUNT_KEY']
)
pt_std_dest = fsspec.get_mapper(
    f"az://fia/daymet/fia_40yr_monthly_std_devs.zarr",
    account_name="usfs",
    account_key=os.environ['AZURE_ACCOUNT_KEY']
)

In [7]:
df = pd.read_csv('./fia_no_pltcn.csv')
train_pts = df.where(df['INVYR'] >= 2015).dropna()
x, y = reproject(train_pts.LON.tolist(), train_pts.LAT.tolist())

In [8]:
mn = xr.open_zarr(mean_src, chunks={'month':1, 'x': 1250, 'y': 1250})
mn

/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/backends/api.py:539: UserWarning: Specified Dask chunks 1250 would separate Zarr chunk shape 505 for dimension 'y'. This significantly degrades performance. Consider rechunking after loading instead.
  store.get_chunk(k, v, chunks),
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/backends/api.py:539: UserWarning: Specified Dask chunks 1250 would separate Zarr chunk shape 977 for dimension 'x'. This significantly degrades performance. Consider rechunking after loading instead.
  store.get_chunk(k, v, chunks),


<xarray.Dataset>
Dimensions:  (month: 12, x: 7814, y: 8075)
Coordinates:
    lat      (y, x) float32 dask.array<chunksize=(1250, 1250), meta=np.ndarray>
    lon      (y, x) float32 dask.array<chunksize=(1250, 1250), meta=np.ndarray>
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * x        (x) float32 -4.56e+06 -4.559e+06 -4.558e+06 ... 3.252e+06 3.253e+06
  * y        (y) float32 4.984e+06 4.983e+06 4.982e+06 ... -3.089e+06 -3.09e+06
Data variables:
    prcp     (month, y, x) float32 dask.array<chunksize=(1, 1250, 1250), meta=np.ndarray>
    swe      (month, y, x) float32 dask.array<chunksize=(1, 1250, 1250), meta=np.ndarray>
    tmax     (month, y, x) float32 dask.array<chunksize=(1, 1250, 1250), meta=np.ndarray>
    tmin     (month, y, x) float32 dask.array<chunksize=(1, 1250, 1250), meta=np.ndarray>
    vp       (month, y, x) float32 dask.array<chunksize=(1, 1250, 1250), meta=np.ndarray>

In [9]:
mn.sel(
    x=xr.DataArray(x, dims='idx', coords={'idx': train_pts['INDEX'].tolist()}),
    y=xr.DataArray(y, dims='idx', coords={'idx': train_pts['INDEX'].tolist()}),
    method="nearest"
).drop_vars(['lat', 'lon', 'x', 'y']).to_zarr(pt_means_dest)

In [10]:
std = xr.open_zarr(std_src, chunks={'month':1, 'x': 1250, 'y': 1250})
std

/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/backends/api.py:539: UserWarning: Specified Dask chunks 1250 would separate Zarr chunk shape 505 for dimension 'y'. This significantly degrades performance. Consider rechunking after loading instead.
  store.get_chunk(k, v, chunks),
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/backends/api.py:539: UserWarning: Specified Dask chunks 1250 would separate Zarr chunk shape 977 for dimension 'x'. This significantly degrades performance. Consider rechunking after loading instead.
  store.get_chunk(k, v, chunks),


<xarray.Dataset>
Dimensions:  (month: 12, x: 7814, y: 8075)
Coordinates:
    lat      (y, x) float32 dask.array<chunksize=(1250, 1250), meta=np.ndarray>
    lon      (y, x) float32 dask.array<chunksize=(1250, 1250), meta=np.ndarray>
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * x        (x) float32 -4.56e+06 -4.559e+06 -4.558e+06 ... 3.252e+06 3.253e+06
  * y        (y) float32 4.984e+06 4.983e+06 4.982e+06 ... -3.089e+06 -3.09e+06
Data variables:
    prcp     (month, y, x) float32 dask.array<chunksize=(1, 1250, 1250), meta=np.ndarray>
    swe      (month, y, x) float32 dask.array<chunksize=(1, 1250, 1250), meta=np.ndarray>
    tmax     (month, y, x) float32 dask.array<chunksize=(1, 1250, 1250), meta=np.ndarray>
    tmin     (month, y, x) float32 dask.array<chunksize=(1, 1250, 1250), meta=np.ndarray>
    vp       (month, y, x) float32 dask.array<chunksize=(1, 1250, 1250), meta=np.ndarray>

In [13]:
std.sel(
    x=xr.DataArray(x, dims='idx', coords={'idx': train_pts['INDEX'].tolist()}),
    y=xr.DataArray(y, dims='idx', coords={'idx': train_pts['INDEX'].tolist()}),
    method="nearest"
).drop_vars(['lat', 'lon', 'x', 'y']).to_zarr(pt_std_dest)

In [14]:
check = xr.open_zarr(pt_std_dest)
check

<xarray.Dataset>
Dimensions:  (idx: 198225, month: 12)
Coordinates:
  * idx      (idx) float64 2.0 5.0 8.0 11.0 ... 9.811e+05 9.811e+05 9.811e+05
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    prcp     (month, idx) float32 dask.array<chunksize=(1, 198225), meta=np.ndarray>
    swe      (month, idx) float32 dask.array<chunksize=(1, 198225), meta=np.ndarray>
    tmax     (month, idx) float32 dask.array<chunksize=(1, 198225), meta=np.ndarray>
    tmin     (month, idx) float32 dask.array<chunksize=(1, 198225), meta=np.ndarray>
    vp       (month, idx) float32 dask.array<chunksize=(1, 198225), meta=np.ndarray>